In [104]:
import gurobipy as gp
from gurobipy import GRB
import tomllib as tml
import numpy as np

Options looks like:
```
options = {
    "WLSACCESSID": "********-****-****-****-************",
    "WLSSECRET": "********-****-****-****-************",
    "LICENSEID": _____,
}
```

In [105]:
# get gurobi credentials
options = tml.load(open("license.toml", "rb"))

In [106]:
# establish env (must close)
env = gp.Env(params=options)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2527858
Academic license 2527858 - for non-commercial use only - registered to mb___@ur.rochester.edu


## Test with MPS File
I made a MPS file by solving LP.mod (written by Quan Luu) with GLPK for Windows.

In [107]:
m = gp.read("model.mps", env=env)
m.reset()
m.optimize()

Read MPS format model from file model.mps
Reading time = 0.00 seconds
LP: 757 rows, 729 columns, 2160 nonzeros
Discarded solution information
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2527858 - for non-commercial use only - registered to mb___@ur.rochester.edu
Optimize a model with 757 rows, 729 columns and 2160 nonzeros
Model fingerprint: 0xcc557f7f
Variable types: 0 continuous, 729 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-03, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective -0.0927000
Presolve removed 27 rows and 0 columns
Presolve time: 0.01s
Presolved: 730 rows, 729 columns, 2160 nonzeros
Variable types: 0 continuous, 729 integer (729

I'm not sure this tells us much. Check `glpk_out.txt`, it has the full output of this solution. 
Notable slice:
```
730 rows, 729 columns, 2160 non-zeros
      0: obj =  -4.657500000e-01 inf =   1.000e+01 (2)
      5: obj =  -1.523000000e-01 inf =   0.000e+00 (0)
*   224: obj =   6.790500000e-01 inf =   2.065e-14 (0) 1
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+   224: mip =     not found yet <=              +inf        (1; 0)
+   224: >>>>>   6.790500000e-01 <=   6.790500000e-01   0.0% (1; 0)
+   224: mip =   6.790500000e-01 <=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOLUTION FOUND
Time used:   0.0 secs
Memory used: 1.9 Mb (1980226 bytes)
STATES:
[1 2 3]   [10 11 12]   [19 20 21]
[4 5 6] , [13 14 15] , [22 23 24].
[7 8 9]   [16 17 18]   [25 26 27]

BUCKETS:
Bucket 5: 1 2 3 4 5 6 7 8 9
Bucket 11: 11 12 19 21
Bucket 13: 10 13
Bucket 14: 14 15
Bucket 17: 16 17 18 25 26 27
Bucket 23: 20 22 23 24
```

## Problem Setup
I'm going to try and convert this outright to a Gurobi model.

In [108]:
# establish model (must close)
model = gp.Model(env=env)


Generating the probability matrix.

In [109]:
def normpdf(x: float, mean: float, std: float) -> float:
  var = float(std)**2
  denom = (2*np.pi*var)**.5
  num = np.exp(-(float(x)-float(mean))**2/(2*var))
  return num/denom

def highlow(ind):
  if ind == 0 or ind == 2:
    return 0.4
  else:
    return 0.2

def gen_state_prob(num_traits: int, num_states: int):
  mean = (num_states-1) / 2
  std = mean / 1.25

  state_prob = np.zeros(tuple([num_states] * num_traits), dtype=np.float64)
  for inds in np.ndindex(state_prob.shape):
    prob = 1
    for ind in inds:
      prob *= normpdf(ind, mean, std)
      # prob *= highlow(ind)
    
    state_prob[inds] = prob

  state_prob = state_prob / np.sum(state_prob)

  return state_prob.flatten()

Generating the reward matrix.

In [110]:
def unnumerize(num_traits: int, num_states: int, action: int):
  ufaction = []
  while action > 0:
    ufaction.insert(0, action % num_states)
    action = action // num_states

  while len(ufaction) < num_traits:
    ufaction.insert(0, 0)

  return ufaction

def reward_fn(param: tuple[float, float], state, action):
  
  l1dist = 0
  for s, a in zip(state, action):
    l1dist += abs(s - a)

  return param[0] - param[1] * l1dist

def reward_matrix(num_traits: int, num_states: int, reward_param: tuple[float, float]):
  total_states = num_states**num_traits
  res = np.array([[0 for _ in range(total_states)] for _ in range(total_states)], dtype=np.float64)
  for x in range(total_states):
    for y in range(total_states):
      s1 = unnumerize(num_traits, num_states, x)
      s2 = unnumerize(num_traits, num_states, y)

      res[x, y] = reward_fn(reward_param, s1, s2)

  return res

Generating the neighbor list

In [111]:
def is_adj(num_traits: int, num_states: int, i: int, j: int):
  s1 = unnumerize(num_traits, num_states, i)
  s2 = unnumerize(num_traits, num_states, j)

  l1dist = 0
  for s, a in zip(s1, s2):
    l1dist += abs(s - a)

  return l1dist == 1

def neighbor_lst(num_traits: int, num_states: int):
  total_states = num_states**num_traits
  res = [[] for _ in range(total_states)]
  for x in range(total_states):
    for y in range(x, total_states):
      if is_adj(num_traits, num_states, x, y):
        res[x].append(y)
        res[y].append(x)

  return res

# def adj_matrix(num_traits: int, num_states: int):
#   total_states = num_states**num_traits
#   res = np.array([[0 for _ in range(total_states)] for _ in range(total_states)])
#   for x in range(total_states):
#     for y in range(total_states):
#       res[x, y] = 1 if is_adj(num_traits, num_states, x, y) else 0

#   return res

In [112]:
# parameters
t = 3
n_per_t = 3
n = n_per_t**t
k = 6
reward_param = (1, 0.3)

V = np.asarray([i for i in range(n)])

# state_prob = gen_state_prob(t, n_per_t)
state_prob = np.full(27, 1/27, dtype=np.float64)

reward = reward_matrix(t, n_per_t, reward_param)

# adj = adj_matrix(t, n_per_t)
neighbor = neighbor_lst(t, n_per_t)

In [113]:
# Hess variables
# x[i, j] = 1 iff state i is in bucket j
# NOTE: "bucket j" means that action j is played in that bucket
x = model.addVars(V, V, vtype=GRB.BINARY)

# cut egde variables
# y[i, j] = 1 iff edge {i, j} is cut
y = model.addVars(V, V, vtype=GRB.BINARY)

# flow variables
# f[i, j] = the flow from state i to state j
# NOTE: f[i, i] = the size of the bucket
f = model.addVars(V, V)

In [114]:
# state objective
# GPL: maximize EP: sum{i in V} PROB[i] * sum{j in V} x[i, j] * REWARD[i, j];
# gp.quicksum( prob[i] * x[i][j] * reward[i][j] for i in V for j in V )
objective = gp.quicksum( gp.quicksum( (state_prob[i] * x[i,j] * reward[i][j]) for j in V) for i in V ) * (n**2 + 1) * 10 - gp.quicksum( y[i, j] for i in V for j in V )
model.setObjective(objective, GRB.MAXIMIZE)

In [115]:
# add constraints

# /* there are exactly k buckets */
# kBucketConstr: sum{j in V} x[j, j] = k;
k_bucket = gp.quicksum( (x[j,j]) for j in V ) == k
model.addConstr(k_bucket)

# /* a state can only belong to one bucket */
# uniqueBucketConstr{i in V}: sum{j in V} x[i, j] = 1;
unique_bucket = ( gp.quicksum( (x[i,j]) for j in V ) == 1 for i in V )
model.addConstrs(unique_bucket)

# /* a state cannot belong to a non-existant bucket */
# nonexBucketConstr{i in V, j in V}: x[i, j] <= x[j, j];
nonex_bucket = ( (x[i,j] <= x[j,j]) for i in V for j in V )
model.addConstrs(nonex_bucket)

model.update()


In [116]:
M = n - k + 1

# /* cut edge constraints */
# edge {i, j} is cut if i and j are not adjacent.
cut_edge_not_adj = ( y[i, j] == 1 for i in V for j in np.setdiff1d(V, np.array(neighbor[i])) )
model.addConstrs(cut_edge_not_adj)

# edge {i, j} is cut if i and j are in different buckets.
cut_edge_diff_bucket = ( y[i, j] >= x[i, l] - x[j, l] for i in V for j in V for l in V)
model.addConstrs(cut_edge_diff_bucket)

# do not send flow across cut edges
cut_edge_flow = ( f[i, j] + f[j, i] <= M * (1 - y[i, j]) for i in V for j in V )
model.addConstrs(cut_edge_flow)

# /* flow constraint */
# if not a root, consume some flow.
# if a root, only send out (so much) flow.
flow = ( gp.quicksum( f[j, i]- f[i, j] for j in neighbor[i] )
      >= 1 - M * x[i, i] for i in V )
model.addConstrs(flow)

model.update()

In [117]:
# multiple solutions
model.Params.PoolSolutions = 5000
model.Params.PoolSearchMode = 2

model.update()

# can we solve?
model.optimize()

Set parameter PoolSolutions to value 5000
Set parameter PoolSearchMode to value 2
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2527858 - for non-commercial use only - registered to mb___@ur.rochester.edu
Optimize a model with 21817 rows, 2187 columns and 62775 nonzeros
Model fingerprint: 0x3a1e3128
Variable types: 729 continuous, 1458 integer (1458 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 18144 rows and 1242 columns
Presolve time: 0.03s
Presolved: 3673 rows, 945 columns, 11151 nonzeros
Variable types: 108 continuous, 837 integer (837 binary)
Found heuristic solution: objective 4272.7777778

Root relaxation: objective 4

### Solution Extraction
This was a little easier than I thought, thanks to Quan's code.

In [118]:
centers = [j for j in V if x[j,j].getAttr("x") == 1]

for j in centers:
    print(f"Bucket {j+1}: ", end="")
    members = [i for i in V if x[i,j].getAttr("x") == 1]
    for i in members:
        print(f"{i+1} ", end="")
    print()


Bucket 4: 1 4 7 13 
Bucket 6: 3 5 6 9 
Bucket 11: 2 10 11 12 20 
Bucket 17: 8 14 16 17 18 26 
Bucket 22: 19 22 25 
Bucket 24: 15 21 23 24 27 


```
STATES:
[1 2 3]   [10 11 12]   [19 20 21]
[4 5 6] , [13 14 15] , [22 23 24].
[7 8 9]   [16 17 18]   [25 26 27]
```

<!-- BUCKETS:
Bucket 5: 1 2 3 4 5 6 7 8 9
Bucket 11: 11 12 19 21
Bucket 13: 10 13
Bucket 14: 14 15
Bucket 17: 16 17 18 25 26 27
Bucket 23: 20 22 23 24

GLPK Output again for comparison. -->

Convert strategy to calculate information content measure.

In [119]:
def convert(num_traits, num_states, num_signals, strat):
  total_states = num_states**num_traits
  small_val = 1e-7
  large_val = 1 - (small_val * (num_signals-1))

  signal_strat = np.full((num_signals, total_states), small_val, dtype=np.float64)

  for i, bucket in enumerate(strat):
    for state in bucket:
      signal_strat[i, state] = large_val

  return signal_strat

def info_measure(num_traits, num_states, num_signals, signal_prob, weighted=True) -> float:
    """Calculates the information content of the signals

    Args:
      signal_prob (np.ndarray): the probabilities of the signals
      weighted (boolean): weighted/unweighted options

    Returns:
      inf (float): total information content measure
      inf_sigs (list): information content by signal
      inf_states (list): information content by state
    """
    total_states = num_states**num_traits
    signal_prob = signal_prob.reshape(num_signals, total_states)

    prob = np.zeros_like(signal_prob)
    for i in range(num_signals):
      for j in range(total_states):
        prob[i, j] = signal_prob[i, j] * state_prob[j]
    prob_sig = [np.sum(prob[i]) for i in range(num_signals)]
    prob = (prob.T / np.sum(prob, axis=1)).T

    inf = 0
    inf_sigs = []
    inf_states = []
    for i in range(num_signals):
      inf_sig = 0
      inf_states.append([])
      for j in range(total_states):
        inf_state = prob[i, j] * np.log(prob[i, j]/state_prob[j])
        inf_sig += inf_state
        
        if weighted:
          inf_state = prob_sig[i] * inf_state

        inf_states[i].append(inf_state)

      if weighted:
        inf_sig = prob_sig[i] * inf_sig

      inf_sigs.append(inf_sig)
      inf += inf_sig

    new_size = [num_signals]
    new_size.extend([num_states] * num_traits)
    inf_states = np.resize(np.array(inf_states), tuple(new_size))

    return inf, inf_sigs, inf_states

def stats(inf, inf_sigstates):
  inf_states = np.sum(inf_sigstates, axis=0)

  print(f"Info measure = {inf}")
  print(f"Info measure by states:")

  for t1 in inf_states:
    for t2 in t1:
      for t3 in t2:
        print(f"{t3:.3f}", end=" ")
      print()
    print()

Calculating the average information content measure of all n results

In [120]:
# n solutions
n_solutions = model.getAttr("SolCount")
print(f"Number of solutions: {n_solutions}")

new_size = [k]
new_size.extend([n_per_t] * t)

total_info = 0
total_info_sigstates = np.zeros(tuple(new_size))
total_w_info = 0
total_w_info_sigstates = np.zeros(tuple(new_size))

for n in range(0, n_solutions):
    model.params.SolutionNumber = n
    # print(f"Solution {n}")
    centers = [j for j in V if x[j,j].getAttr("Xn") == 1]
    strat = []
    for j in centers:
        # print(f"Bucket {j+1}: ", end="")
        members = [i for i in V if x[i,j].getAttr("Xn") == 1]
        #     for i in members:
        #         print(f"{i+1} ", end="")
        #     print()
        # print()
        strat.append(members)

    converted_strat = convert(t, n_per_t, k, strat)
        
    inf, inf_sigs, inf_sigstates = info_measure(t, n_per_t, k, converted_strat, False)

    w_inf, w_inf_sigs, w_inf_sigstates = info_measure(t, n_per_t, k, converted_strat)

    total_info += inf
    total_info_sigstates += inf_sigstates

    total_w_info += w_inf
    total_w_info_sigstates += w_inf_sigstates
    
avg_info = total_info / n_solutions
avg_info_sigstates = total_info_sigstates / n_solutions
avg_w_info = total_w_info / n_solutions
avg_w_info_sigstates = total_w_info_sigstates / n_solutions

print("UNWEIGHTED")
stats(avg_info, avg_info_sigstates)
print()
print("WEIGHTED")
stats(avg_w_info, avg_w_info_sigstates)

Number of solutions: 5000
UNWEIGHTED
Info measure = 10.586776168929758
Info measure by states:
0.494 0.378 0.463 
0.372 0.365 0.355 
0.466 0.365 0.473 

0.367 0.360 0.350 
0.355 0.245 0.351 
0.348 0.352 0.356 

0.484 0.372 0.473 
0.371 0.372 0.373 
0.480 0.386 0.461 


WEIGHTED
Info measure = 1.6827569209450635
Info measure by states:
0.068 0.061 0.066 
0.061 0.062 0.060 
0.066 0.061 0.067 

0.061 0.061 0.059 
0.061 0.053 0.061 
0.059 0.061 0.060 

0.068 0.061 0.067 
0.061 0.062 0.061 
0.067 0.061 0.066 



In [121]:
# closing these objects for best practice

model.close()
m.close()
env.close()